In [ ]:
import os
import glob
import json
import math
import random
import re
from collections import Counter
import networkx as nx
import pdfplumber
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# Set the OpenAI API key from environment variable
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# -- Document Loading ------------------------------------------------------

def load_docs_from_dir(dir_path):
    """
    Load all .pdf files from a directory and extract their full text.
    Returns a list of strings, one per PDF.
    """
    docs = []
    filenames = []
    for filepath in glob.glob(os.path.join(dir_path, '*.pdf')):
        try:
            text_pages = []
            with pdfplumber.open(filepath) as pdf:
                for page in pdf.pages:
                    text_pages.append(page.extract_text() or "")
            full_text = "\n".join(text_pages)
            docs.append(full_text)
            filenames.append(os.path.basename(filepath))
        except Exception as e:
            # skip unreadable PDFs
            print(f"Warning: could not load {filepath}: {e}")
    return docs, filenames

# -- 1. Extract topics and key concepts -----------------------------------

def extract_json_from_text(text):
    """Helper function to extract JSON content from markdown code blocks or raw text"""
    # Try to extract JSON from markdown code blocks
    json_match = re.search(r'```(?:json)?\s*(.*?)\s*```', text, re.DOTALL)
    if json_match:
        return json_match.group(1).strip()
    
    # If no code blocks, try to find JSON-like structures
    json_match = re.search(r'(\{.*\})', text, re.DOTALL)
    if json_match:
        return json_match.group(1).strip()
    
    # Return original text if no JSON structure found
    return text

def extract_concepts_from_docs(doc_texts, filenames, model="gpt-4o"):  
    """
    Call OpenAI API to extract topics and key concepts from each document.
    Returns list of dicts with keys 'topics' and 'key_concepts'.
    """
    extractions = []
    for i, text in enumerate(doc_texts):
        filename = filenames[i] if i < len(filenames) else f"doc_{i}"
        
        prompt = (
            "Extract high-level topics and key concepts from the following document. "
            f"Return JSON with keys 'topics' and 'key_concepts'.\n\n{text}"
        )
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an expert summarizer. Return your response as a JSON object with 'topics' and 'key_concepts' as arrays."},
                {"role": "user", "content": prompt}
            ],
        )
        
        output = response.choices[0].message.content
        print("Raw output:", output)
        
        try:
            # Clean and extract JSON from the output
            json_str = extract_json_from_text(output)
            data = json.loads(json_str)
            
            # Ensure we have the expected keys
            if "topics" in data and "key_concepts" in data:
                extraction = {
                    "filename": filename,
                    "topics": data["topics"],
                    "key_concepts": data["key_concepts"]
                }
                extractions.append(extraction)
                print(f"Successfully extracted {len(data['topics'])} topics and {len(data['key_concepts'])} key concepts from {filename}")
            else:
                print(f"Warning: Parsed JSON doesn't have expected keys: {data.keys()}")
                extractions.append({
                    "filename": filename,
                    "topics": [], 
                    "key_concepts": []
                })
                
        except json.JSONDecodeError as e:
            print(f"Failed to parse JSON: {e}")
            extractions.append({
                "filename": filename,
                "topics": [], 
                "key_concepts": []
            })
    
    return extractions

# -- 2. Construct the concept graph ----------------------------------------

def build_concept_graph(extractions, eps=1e-6):
    """
    Build a unified graph G where nodes are topics + key concepts and
    edges weighted by log(freq+eps) based on co-occurrence in docs.
    Returns: G (nx.Graph), topic_nodes, kc_nodes
    """
    freq = Counter()
    all_topics, all_kcs = set(), set()

    for ex in extractions:
        nodes = ex["topics"] + ex["key_concepts"]
        for i in range(len(nodes)):
            for j in range(i+1, len(nodes)):
                u, v = sorted((nodes[i], nodes[j]))
                freq[(u, v)] += 1
        all_topics.update(ex["topics"])
        all_kcs.update(ex["key_concepts"])

    G = nx.Graph()
    for (u, v), f in freq.items():
        weight = math.log(f + eps)
        G.add_edge(u, v, weight=weight)

    print(f"Graph built with {len(all_topics)} topics and {len(all_kcs)} key concepts")
    return G, all_topics, all_kcs

# -- Helpers for sampling --------------------------------------------------

def softmax(weights):
    exps = [math.exp(w) for w in weights]
    s = sum(exps) or 1.0
    return [e/s for e in exps]


def random_walk(G, start, steps):
    """
    Random walk on graph G for given steps from 'start',
    with transition probabilities via softmax over edge weights.
    """
    path = [start]
    current = start
    for _ in range(steps):
        nbrs = list(G[current])
        if not nbrs:
            break
        weights = [G[current][n]['weight'] for n in nbrs]
        probs = softmax(weights)
        current = random.choices(nbrs, probs)[0]
        path.append(current)
    return path

# -- 3. Concept combination sampling ---------------------------------------

def sample_concept_combinations(
    G, topic_nodes, kc_nodes,
    num_samples=100,
    topic_walk_steps=(1, 2),
    kc_walk_steps=(3, 4)
):
    """
    Generate sampled sets of topics and key concepts via multi-stage random walks.
    Returns list of dicts: {'topics': set, 'key_concepts': set}
    """
    # Safety check
    if not topic_nodes:
        print("Error: No topics found. Cannot sample combinations.")
        return []
        
    G_topic = G.subgraph(topic_nodes)
    G_topic_kc = G.subgraph(topic_nodes | kc_nodes)
    G_kc = G.subgraph(kc_nodes)
    samples = []

    topics_list = list(topic_nodes)
    print(f"Sampling from {len(topics_list)} topics")
    
    for _ in range(num_samples):
        t0 = random.choice(topics_list)
        t_steps = random.choice(topic_walk_steps)
        topic_path = random_walk(G_topic, t0, t_steps)
        sampled_topics = set(topic_path)

        kc_cands = [nbr for t in sampled_topics for nbr in G_topic_kc[t] if nbr in kc_nodes]
        if kc_cands:
            k0 = random.choice(kc_cands)
            k_steps = random.choice(kc_walk_steps)
            kc_path = random_walk(G_kc, k0, k_steps)
            sampled_kcs = set(kc_path)
        else:
            sampled_kcs = set()

        samples.append({
            "topics": list(sampled_topics),  # Convert sets to lists for JSON serialization
            "key_concepts": list(sampled_kcs)
        })
    
    return samples

# -- 4. Question generation ------------------------------------------------

def generate_questions_for_samples(combos, docs, extractions, model="gpt-4o"):  
    """
    For each sampled combo, pick two docs via Jaccard on concept sets,
    then call LLM to generate questions.
    Returns list of dicts: {'sample': combo, 'questions': [...]}.
    """
    doc_concepts = [set(ex['topics'] + ex['key_concepts']) for ex in extractions]
    # doc_concepts= doc_concepts[:1]
    results = []
    max_samples = 100
    # combos = combos[:max_samples]
    for i, combo in enumerate(combos):
        combo_id = f"combo_{i+1}"
        kg = set(combo['topics']) | set(combo['key_concepts'])
        sims = []
        for idx, dc in enumerate(doc_concepts):
            inter = kg & dc
            union = kg | dc
            sims.append((len(inter) / (len(union) or 1), idx))
        sims.sort(reverse=True)
        top_idxs = [i for _, i in sims[:2]]
        refs = [docs[i] for i in top_idxs]
        ref_files = [extractions[i]["filename"] for i in top_idxs]
        System_prompt =f"""
Each question must follow these instructions:
Model a Chemistry Scenario: Start from a real-world or idealized setup. Avoid abstract chemistry problems or purely conceptual statements.
Target a Solvable Quantity: Ask for a clear symbolic expression of a physical variable (e.g., tension, acceleration, energy).
Force Multi-Step Reasoning: Ensure the question involves a sequence of chemistry laws, transformations, and derivations to reach the answer.
Avoid Redundancy: Exclude extraneous details or variables that do not impact the final solution.
Be Unique: Do not rephrase standard textbook problems; ensure originality and complexity.
Single solution: Expect a single symbolic expression, unambiguous, presented in LaTeX. Multiple equivalent algebraic forms are allowed. No equations or floating-point approximations.
Use rigorous, concise phrasing.
Avoid colloquial or ambiguous terminology.
Units must be consistent; symbols should follow standard notation.
"""
        prompt = (
            f"Generate a set of difficult chemistry questions based on the following:\n"
            
            f"Topics: {combo['topics']}\n"
            f"Key Concepts: {combo['key_concepts']}\n"
            f"Reference Doc 1:\n{refs[0]}\n"
        )
        if len(refs) > 1:
            prompt += f"Reference Doc 2:\n{refs[1]}\n"
        prompt += "Return a JSON array of questions."

        # from ollama import chat
        # from ollama import ChatResponse

        # response: ChatResponse = chat(model='qwen3:8b', 
        #                                messages=[
        #         {"role": "system", "content": System_prompt},
        #         {"role": "user", "content": prompt}
        #     ])
        # # print(response['message']['content'])
        # # or access fields directly from the response object
        # print(response.message.content)
        # content = response.message.content
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": System_prompt},
                {"role": "user", "content": prompt}
            ]
        )
        content = response.choices[0].message.content
        
        try:
            # Clean and extract JSON from the output
            json_str = extract_json_from_text(content)
            questions = json.loads(json_str)
            if not isinstance(questions, list):
                # If the output is an object with a questions key
                if isinstance(questions, dict) and "questions" in questions:
                    questions = questions["questions"]
                else:
                    questions = [str(questions)]
        except json.JSONDecodeError:
            questions = [content]

        results.append({
            "id": combo_id,
            "topics": combo['topics'],
            "key_concepts": combo['key_concepts'],
            "reference_files": ref_files,
            "questions": questions
        })
        max_samples-=1
        if max_samples == 0:
            break


    return results

# -- Save outputs to files ------------------------------------------------

def save_extractions(extractions, output_file="document_extractions.json"):
    """Save the extracted topics and key concepts for each document"""
    # Ensure the extractions are serializable (convert sets to lists)
    serializable_extractions = []
    for ex in extractions:
        serializable_extractions.append({
            "filename": ex["filename"],
            "topics": list(ex["topics"]),
            "key_concepts": list(ex["key_concepts"])
        })
    
    with open(output_file, "w") as f:
        json.dump(serializable_extractions, f, indent=2)
    
    print(f"Saved document extractions to {output_file}")

def save_questions_with_topics(questions, output_file="questions_with_topics.json"):
    """Save the generated questions with their topic combinations"""
    with open(output_file, "w") as f:
        json.dump(questions, f, indent=2)
    
    print(f"Saved questions with topic combinations to {output_file}")

# -- Main Execution --------------------------------------------------------

if __name__ == "__main__":
    # Create output directory if it doesn't exist
    output_dir = "output_new"
    os.makedirs(output_dir, exist_ok=True)
    
    # adjust this path to where your .pdf docs live
    docs_dir = "docs_new/"
    
    print("Loading documents...")
    docs, filenames = load_docs_from_dir(docs_dir)
    print(f"Loaded {len(docs)} documents")
    
    if not docs:
        print("No documents found. Please check the docs directory.")
        exit(1)
    
    # 1) Extract topics & KCs
    print("\nExtracting topics and key concepts...")
    extractions = extract_concepts_from_docs(docs, filenames)
    
    # Save extractions to file
    save_extractions(extractions, os.path.join(output_dir, "document_extractions_chemistry.json"))
    
    # Verify we have valid extractions
    valid_extractions = [ex for ex in extractions if ex["topics"] or ex["key_concepts"]]
    if not valid_extractions:
        print("No valid topics or key concepts extracted. Check your data and API responses.")
        exit(1)
    
    # 2) Build graph
    print("\nBuilding concept graph...")
    G, topic_nodes, kc_nodes = build_concept_graph(extractions)
    
    if not topic_nodes:
        print("No topics found in the graph. Cannot proceed.")
        exit(1)
    
    # 3) Sample combinations
    print("\nSampling concept combinations...")
    combos = sample_concept_combinations(G, topic_nodes, kc_nodes, num_samples=50)  # Reduced for testing
    
    if not combos:
        print("Failed to generate concept combinations.")
        exit(1)
    
    # Save topic combinations
    with open(os.path.join(output_dir, "topic_combinations_chemistry.json"), "w") as f:
        json.dump(combos, f, indent=2)
    
    # 4) Generate questions
    # print("\nGenerating questions for each combination...")
    # q_outputs = generate_questions_for_samples(combos, docs, extractions)
    
    # # Save questions with topics
    # save_questions_with_topics(q_outputs, os.path.join(output_dir, "questions_with_topics.json"))
    
    # # Display results
    # print("\n===== GENERATED QUESTIONS =====")
    # for idx, out in enumerate(q_outputs, 1):
    #     print(f"\nSample {idx}:")
    #     print(f"Topics: {out['topics']}")
    #     print(f"Key Concepts: {out['key_concepts']}")
    #     print(f"Reference Files: {out['reference_files']}")
    #     print("Questions:")
    #     for q in out['questions']:
    #         print(f" - {q}")
    
    # print(f"\nAll outputs saved to directory: {output_dir}") 

# if __name__ == "__main__":
#     # Create output directory if it doesn't exist
#     # Create output directory if it doesn't exist
#     output_dir = "output_apple"
#     os.makedirs(output_dir, exist_ok=True)
    
#     # adjust this path to where your .pdf docs live
#     docs_dir = "docs/"
    
#     print("Loading documents...")
#     docs, filenames = load_docs_from_dir(docs_dir)
#     print(f"Loaded {len(docs)} documents")
#     document_extractions = []
#     with open("output_apple/document_extractions.json", "r") as f:
#         document_extractions = json.load(f)
#     for doc in document_extractions:
#         print(doc)
#         break

#     topic_combinations = []
#     with open("output_apple/topic_combinations.json", "r") as f:
#         topic_combinations = json.load(f)
#     for combo in topic_combinations:
#         print(combo)
#         break

#     # generate questions 
#     print("Generating questions...")
#     q_outputs = generate_questions_for_samples(topic_combinations, docs, document_extractions)

#     # Save questions with topics
#     save_questions_with_topics(q_outputs, os.path.join(output_dir, "questions_with_topics.json"))
    
#     # Display results
#     print("\n===== GENERATED QUESTIONS =====")
#     for idx, out in enumerate(q_outputs, 1):
#         print(f"\nSample {idx}:")
#         print(f"Topics: {out['topics']}")
#         print(f"Key Concepts: {out['key_concepts']}")
#         print(f"Reference Files: {out['reference_files']}")
#         print("Questions:")
#         for q in out['questions']:
#             print(f" - {q}")
    
#     print(f"\nAll outputs saved to directory: {output_dir}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Loading documents...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Loaded 28 documents

Extracting topics and key concepts...
Raw output: ```json
{
  "topics": [
    "Glycomics and Clinical Applications",
    "Glycan Fragmentation Mechanisms",
    "Collision-Induced Dissociation (CID)",
    "Mass Spectrometry Techniques",
    "Anomeric and Linkage Memory",
    "Hexose and Fucose Migration",
    "Structural Analysis Techniques"
  ],
  "key_concepts": [
    "Glycan fragmentation in glycomics",
    "Collision-induced dissociation (CID) mechanism",
    "Hexose and fucose migration",
    "Ion mobility spectrometry (IMS)",
    "Gas-phase infrared (IR) spectroscopy",
    "Anomeric memory in glycan fragments",
    "Linkage and ring-size memory in glycans",
    "Charge migration in glycan ions",
    "Clinical relevance of glycan profiling",
    "Impact of ion polarity on fragmentation",
    "Influence of adducts and derivatization on glycan stability",
    "Challenges in distinguishing glycan isomers",
    "Use of mass spectrometry for glycan analysis",
    "I

In [3]:
combos = sample_concept_combinations(G, topic_nodes, kc_nodes, num_samples=50)  # Reduced for testing

if not combos:
    print("Failed to generate concept combinations.")
    exit(1)

# Save topic combinations
with open(os.path.join(output_dir, "topic_combinations_chemistry.json"), "w") as f:
    json.dump(combos, f, indent=2)


Sampling from 153 topics


### download data from csv

In [1]:
import os
import argparse
import pandas as pd
import requests
from urllib.parse import urlparse, unquote

def download_files(
    csv_path="Research Papers - Chemistry_New.csv",
    output_dir="docs_new"
) -> None:
    """
    Reads a CSV with a 'Link' column and downloads each URL to output_dir,
    saving each file with a .pdf extension.
    """
    # Load CSV
    df = pd.read_csv(csv_path)
    if 'URL' not in df.columns:
        raise ValueError("CSV must contain a 'URL' column.")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    total = len(df['URL'].dropna())
    for idx, url in enumerate(df['URL'].dropna(), start=1):
        try:
            # Parse URL and derive a safe filename
            parsed = urlparse(url)
            raw_name = os.path.basename(parsed.path)
            raw_name = unquote(raw_name)  # decode URL-encoded characters

            if raw_name:
                name, ext = os.path.splitext(raw_name)
                # If there's no .pdf extension, force it
                filename = f"{idx}.pdf" if ext.lower() != '.pdf' else raw_name
            else:
                filename = f"{idx}.pdf"

            out_path = os.path.join(output_dir, filename)

            # Stream download
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(out_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)

            print(f"[{idx}/{total}] Downloaded → {filename}")

        except Exception as e:
            print(f"[{idx}/{total}] FAILED  → {url}\n    {e}")

if __name__ == "__main__":
    download_files()


/Users/suyashsethia/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[1/47] Downloaded → 1860-5397-20-1.pdf
[2/47] Downloaded → 1860-5397-20-17.pdf
[3/47] Downloaded → 1860-5397-20-19.pdf
[4/47] Downloaded → 1860-5397-21-87.pdf
[5/47] Downloaded → 1860-5397-20-21.pdf
[6/47] Downloaded → 1860-5397-20-259.pdf
[7/47] Downloaded → 7.pdf
[8/47] Downloaded → 1860-5397-20-31.pdf
[9/47] Downloaded → 1860-5397-20-135.pdf
[10/47] Downloaded → 1860-5397-20-241.pdf
[11/47] Downloaded → 1860-5397-20-155.pdf
[12/47] Downloaded → 1860-5397-20-135.pdf
[13/47] FAILED  → https://www.mdpi.com/2304-6740/12/2/51/pdf
    403 Client Error: Forbidden for url: https://www.mdpi.com/2304-6740/12/2/51/pdf
[14/47] FAILED  → https://www.mdpi.com/2304-6740/12/3/92/pdf
    403 Client Error: Forbidden for url: https://www.mdpi.com/2304-6740/12/3/92/pdf
[15/47] FAILED  → https://www.mdpi.com/2304-6740/12/1/4/pdf
    403 Client Error: Forbidden for url: https://www.mdpi.com/2304-6740/12/1/4/pdf
[16/47] FAILED  → https://www.mdpi.com/2304-6740/13/3/91/pdf
    403 Client Error: Forbidden f

## Author - Critique Model

In [ ]:
import os
import re
import uuid
import json
from dotenv import load_dotenv
import os
import logging
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from .env
load_dotenv()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

def author_critique_loop(
    topics: list[str],
    concepts: list[str],
    guidelines: str,
    few_shot_examples: list[dict],
    max_turns: int = 2,
    level: str = "advanced"
) -> list[dict]:
    """
    Runs an iterative author-critic loop to generate and refine chemistry questions.

    Logs the entire multi-turn conversation (author + critic) to a single file.

    Returns a list of dicts with:
        unique_id: str,
        question: str,
        topics: list[str],
        concepts: list[str],
        file_conversation_log: str  # path to the full conversation log
    """
    # Create a unique run-level conversation log
    run_id = uuid.uuid4().hex
    os.makedirs('logs', exist_ok=True)
    conv_log_path = f"logs/{run_id}.log"
    conv_logger = open(conv_log_path, 'w')

    def log(msg: str):
        conv_logger.write(msg + "\n")
        conv_logger.flush()

    # Initialize LLMs (temperature locked to 1.0)
    author_llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=1.0)
    critic_llm = ChatOpenAI(model_name="gpt-4.1", temperature=1.0)
    gemini_api = os.getenv("GEMINI_API_KEY")
    # critic_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=1.0,google_api_key=gemini_api)
    # Build Few-Shot prompt for author
    example_prompt = PromptTemplate(
        input_variables=["example"],
        template="Example:\n{example}\n---"
    )
    author_fs_prompt = FewShotPromptTemplate(
        examples=few_shot_examples,
        example_prompt=example_prompt,
        prefix=(
            "You are a Chemistry education specialist. Generate novel {level} chemistry questions. "
            "Each question must start with <question> and end with </question>. Here are examples:"
        ),
        suffix=(
            "Now, based on the topics: {topics}, key concepts: {concepts}, and guidelines: {guidelines}, "
            "produce list of new, high-quality chemistry questions, each wrapped in <question>...</question> tags."
        ),
        input_variables=["topics", "concepts", "guidelines","level"],
    )
    author_chain = LLMChain(llm=author_llm, prompt=author_fs_prompt)

    # Build Critic prompt
    critic_prompt = PromptTemplate(
        input_variables=["questions", "guidelines"],
        template=(
            "You are a chemistry assessment expert. Critique the following questions:\n"
            "{questions}\n"
            "Evaluate them against these guidelines:\n{guidelines}\n"
            "Provide concise, actionable feedback on how to improve."
        ),
    )
    critic_chain = LLMChain(llm=critic_llm, prompt=critic_prompt)

    # Turn 0: Generation
    log("=== Turn 0: Author generates questions ===")
    questions_text = author_chain.run(
        topics=topics, concepts=concepts, guidelines=guidelines, level=level
    )
    log(questions_text)

    # Turn 0: Critic feedback
    log("=== Turn 0: Critic feedback ===")
    feedback = critic_chain.run(questions=questions_text, guidelines=guidelines)
    log(feedback)

    # Refinement turns
    for turn in range(1, max_turns):
        log(f"=== Turn {turn}: Author refines questions ===")
        refine_prompt = PromptTemplate(
            input_variables=["questions", "feedback"],
            template=(
                "Refine these questions based on the feedback:\n{questions}\n"
                "Feedback:\n{feedback}\n"
                "Return an improved numbered list of chemistry questions, each wrapped in <question>...</question> tags."
            ),
        )
        refine_chain = LLMChain(llm=author_llm, prompt=refine_prompt)
        questions_text = refine_chain.run(questions=questions_text, feedback=feedback)
        log(questions_text)
        if turn == max_turns - 1:
            log("=== Final questions generated ===")
            break
        log(f"=== Turn {turn}: Critic feedback ===")
        feedback = critic_chain.run(questions=questions_text, guidelines=guidelines)
        log(feedback)

    # Close the conversation log
    conv_logger.close()

    # Parse final questions into structured entries
    entries: list[dict] = []
    pattern = re.compile(r'<question>(.*?)</question>', re.DOTALL)
    for match in pattern.findall(questions_text):
        question_body = match.strip()
        uid = uuid.uuid4().hex
        entries.append({
            "unique_id": uid,
            "level": level,
            "topics": topics,
            "concepts": concepts,
            "question": question_body,
            "file_conversation_log": conv_log_path
        })
        print(f"Generated question: {question_body}")

    return entries


if __name__ == "__main__":
    # Chemistry-specific example usage
    # topics = [
    #     "Thermodynamics of Chemical Reactions",
    #     "Organic Reaction Mechanisms",
    #     "Coordination Chemistry",
    #     "Electrochemical Cells"
    # ]
    # concepts = [
    #     "Use of enthalpy, entropy, and Gibbs free energy to predict reaction spontaneity.",
    #     "Mechanistic steps in nucleophilic substitution and elimination reactions.",
    #     "Formation constants and isomerism in transition metal complexes.",
    #     "Calculating cell potential using standard reduction potentials and Nernst equation."
    # ]
    with open("output_new/topic_combinations_chemistry.json", "r") as f:
        topic_combinations = json.load(f)
    
    # make topic combination a list conatainin only the 3rd element
    # topic_combinations = [combo['topics'] for combo in topic_combinations]
    topic_combinations = topic_combinations[:20] # Limit to first 10 for testing
    # guidelines = (
    #     "1. Model a Chemistry Scenario: Start from a real-world or idealized setup."
    #     "2. Expect answers as numerical values with appropriate units or chemical compounds named using official IUPAC nomenclature. "
    #     "3. Force Multi-Step Reasoning: Ensure calculations or mechanistic steps are detailed. "
    #     "4. Avoid Redundancy: Exclude extraneous information that does not affect the answer. "
    #     "5. Be Unique: Do not repeat standard textbook examples; ensure originality and complexity. "
    #     "6. Single solution: Provide one correct numerical value or compound name. "
    #     "7. Use rigorous, concise phrasing. "
    #     "8. Avoid colloquial or ambiguous terminology. "
    #     "9. Units must be consistent; chemical nomenclature must follow IUPAC rules."
    #     "10. Questions must not have multiple parts to answer, there should be only 1 solution to each question"
    #     "11. give the final question in inline markdown snippet code using $ instead of # for the purpose of rendering"





    # )
    #make a enum of level 
    levels = ["easy", "medium", "hard"]
    level = "hard"  # Change this to "high school", "college", or "phd" as needed
    if level not in levels:
        raise ValueError(f"Invalid level: {level}. Must be one of {levels}.")
    # Example few-shot examples
    # Note: These examples should be tailored to the specific level of difficulty

    if level == "easy":
        few_shot_examples = [
            {"example": (
                r"""Good: SN1 glycosylation selectivity (one decimal place)  
A 4,6-O-benzylidene-protected α-D-glucopyranosyl trichloroacetimidate donor undergoes SN1 glycosylation in CH₂Cl₂ at 298 K.  DFT gives  
 ΔG‡α = 85.8 kJ·mol⁻¹  
 ΔG‡β = 93.3 kJ·mol⁻¹  

The ratio of product mole fractions is  
 Pα/Pβ = exp[(ΔG‡β – ΔG‡α)/(R T)].  

Calculate the α : β ratio in the form X : 1.  
Report the value of X (α/β) to one decimal place.  """
            )},
            {"example": (
                r"Bad: What is the pH of a solution?"
            )},
        ]
        #easy guidlines
        guidelines = (
    r"1. Model a Chemistry Scenario: Start from a clear but very simple real-world or textbook-style setup."
    r"2. Expect answers as single numerical values with correct units **or** a single compound name in official IUPAC nomenclature. "
    r"3. Single-Step Calculation: The question should require at most one direct substitution into a standard formula or one straightforward fact recall. "
    r"4. Avoid Redundancy: Provide only the data strictly needed for that one calculation or fact. "
    r"5. Be Unique: Even simple, the context should not be copied verbatim from common examples. "
    r"6. Single solution: Provide exactly one correct numerical value or compound name. "
    r"7. Use rigorous, concise phrasing. "
    r"8. Avoid colloquial or ambiguous terminology. "
    r"9. Units must be consistent; chemical nomenclature must follow IUPAC rules. "
    r"10. Questions must not have multiple parts; a single answer is expected. "
    r"11. Solution Formatting: Format equations, numbers and variables using LaTeX with Markdown syntax: "
    r"Replace \\(...\\) with $...$ "
    r"Replace \\[...\\] with $$...$$ "
    r"Example $c = 0.10\\,\\mathrm{mol\\,L^{-1}}$, $\\Delta H = -57\\,\\mathrm{kJ\\,mol^{-1}}$"
)
    elif level == "medium":
        few_shot_examples = [
            {"example": (
                r"""Good:  Half-life of intramolecular cyclization (two significant figures)  
A 4,6-O-benzylidene-protected D-galactopyranosyl oxocarbenium ion cyclizes intramolecularly via O4→C1 with ΔG‡ = 59.4 kJ·mol⁻¹ (convert to J·mol⁻¹ before use) at 298 K.  Using  

 k = (kB T/h) exp(–ΔG‡/(R T))  
 t₁/₂ = (ln 2)/k  

calculate the half-life t₁/₂ in seconds at 298 K.  Report t₁/₂ to two significant figures. """
            )},
            {"example": (
                r"Bad: Define molarity."
            )},
        ]
        #medium guidelines
        guidelines = (
    r"1. Model a Chemistry Scenario: Start from a real-world or idealized setup that may involve a short descriptive context, diagram or simple data table. "
    r"2. Expect answers as numerical values with appropriate units **or** compound names in official IUPAC nomenclature. "
    r"3. Moderate Reasoning: Require 2–3 logical or computational steps — e.g., selecting the correct formula, performing a stoichiometric ratio, or interpreting a simple graph. "
    r"4. Provide only necessary data; remove any fluff that doesn’t influence the outcome. "
    r"5. Be Unique: Avoid rehashing textbook end-of-chapter questions while staying at an intermediate difficulty. "
    r"6. Single solution: Provide one correct numerical value or compound name. "
    r"7. Use rigorous, concise phrasing. "
    r"8. Avoid colloquial or ambiguous terminology. "
    r"9. Units must be consistent; chemical nomenclature must follow IUPAC rules. "
    r"10. Questions must not have multiple parts; produce only one final answer. "
    r"11. Solution Formatting: Format equations, numbers and variables using LaTeX with Markdown syntax: "
    r"Replace \\(...\\) with $...$ "
    r"Replace \\[...\\] with $$...$$ "
    r"Example $K_{\\mathrm{eq}} = 4.0 \\times 10^{-3}$, $pK_a = 2.15$"
)
    elif level == "hard":
        few_shot_examples = [
        {"example": (
            r"""Good: Solvent-dependent rate equality (three significant figures)  
The same donor has activation free energies at 298 K:  
 ΔG‡(CH₂Cl₂) = 96.3 kJ·mol⁻¹  
 ΔG‡(MeCN)   = 89.9 kJ·mol⁻¹  

According to the Eyring equation  
 k = (kB T/h) exp(–ΔG‡/(R T)),  

neglect the temperature variation of the pre-exponential factor (treat kB T/h as equal in both rate constants).  Find the temperature T (in K) at which  
 k_CH₂Cl₂(T) = k_MeCN(298 K).  """
        )},
        {"example": (
            r"Bad: Define enthalpy."
        )},
        ]
        #hard guidelines
        # ----------  (Existing hard-level guidelines remain unchanged)  ----------
        guidelines = (
    r"1. Model a Chemistry Scenario: Start from a real-world or idealized setup."
    r"2. Expect answers as numerical values with appropriate units or chemical compounds named using official IUPAC nomenclature. "
    r"3. Force Multi-Step Reasoning: Ensure calculations or mechanistic steps are detailed. "
    r"4. Avoid Redundancy: Exclude extraneous information that does not affect the answer. "
    r"5. Be Unique: Do not repeat standard textbook examples; ensure originality and complexity. "
    r"6. Single solution: Provide one correct numerical value or compound name. "
    r"7. Use rigorous, concise phrasing. "
    r"8. Avoid colloquial or ambiguous terminology. "
    r"9. Units must be consistent; chemical nomenclature must follow IUPAC rules."
    r"10. Questions must not have multiple parts to answer, there should be only 1 solution to each question"
    r"11. Solution Formatting: Format the equations, numbers and variables using LaTeX with Markdown syntax: "
    r"Replace \\(...\\) with $...$ "
    r"Replace \\[...\\] with $$...$$ "
    r"the questions must be renderable in Markdown, so use $ for inline math and $$ for block math. "
    r"Example:  $F = ma$   and   $$E = mc^2$$."
)
    all_questions = []

    for combo in topic_combinations:
        topics = combo['topics']
        concepts = combo['key_concepts']
        # logger.info(f"Processing combination: {topics} | {concepts}")
        # Run the author–critic loop for this combination
        questions = author_critique_loop(
            topics, concepts, guidelines, few_shot_examples, max_turns=4, level=level
        )
        
        all_questions.append({
            "data": questions
        })

    # Save all questions to a file
    with open("final_questions_chemistry.json", "w") as f:
        json.dump(all_questions, f, indent=2)
    
    # 2) write a Markdown file you can grab directly
    with open("final_questions_chemistry.md", "w") as md:
        for batch in all_questions:
            for q in batch["data"]:
                # Write the question with its unique ID and topics
                md.write(f"### Question ID: {q['unique_id']}\n")
                # Write the question text
                md.write("**Question:**\n")
            # q["question"] already contains things like $\theta$
                md.write(q["question"] + "\n\n")


KeyboardInterrupt: 

In [7]:
    # 2) write a Markdown file you can grab directly
with open("final_questions_chemistry.md", "w") as md:
        for batch in all_questions:
            for q in batch["data"]:
                # Write the question with its unique ID and topics
                md.write(f"### Question ID: {q['unique_id']}\n")
                # Write the question text
                md.write("**Question:**\n")
            # q["question"] already contains things like $\theta$
                md.write(q["question"] + "\n\n")

### converting json to readable txt

In [3]:
data = {
    "question": r"Find the angle $\theta$ such that $\sin \theta = 0.5$."
}

with open("out.json", "w") as f:
    json.dump(data, f, indent=2)

#load the json and print the question
with open("out.json", "r") as f:
    data = json.load(f)
    print(data["question"])  # Output: Find the angle $\theta$ such that $\sin \theta = 0.5$.


Find the angle $\theta$ such that $\sin \theta = 0.5$.


In [4]:
# write the question to a file .md
with open("question.md", "w") as f:
    f.write(data["question"])  # Output: Find the angle $\theta$ such that $\sin \theta = 0.5$.